### 지니 Top 200 차트 

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'

In [4]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
req.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [5]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text)

# 과제 

- 지니차트 첫화면에 50곡중에서 순위,제목,가수,앨범명 을 크롤링하여 데이터 프레임을 만들고 지니차트 (20020414)Top50.csv 파일을 만들기

### 2. 찾으려고 하는 데이터로드 찾기

In [136]:
# 지니는 (table) 
     #       <thead></thead>
     #       <tbody>
      #          <tr></tr>  #이게 우리가 찾아야하는 한줄단위
      #      <tbody>
#</table>

In [137]:
trs = soup.select('tr.list')
len(trs)

50

In [138]:
#  개발자도구에서 찾아주는 셀렉터 (selector)
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

In [139]:
#body-content > div.newest-list > div > table > tbody > tr:nth-child(1)

### 3. 여러개의 데이터중 하나를 선택해서 원하는 정보를 추출 

In [140]:
tr=trs[12]

In [141]:
tr.select_one('.number').get_text()

'13\n                                        \n                                    \n8상승\n\n'

In [142]:
try:
    rank =int(tr.select_one('.number').get_text().split('\n')[0])
except:
    rank=0
rank


13

In [143]:
# 제목 
# 일반적인 노래
try:
    title=tr.select_one('.title.ellipsis').string.strip()
except:
    title=tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()


# 19금 노래 
title

'회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom)'

In [144]:
artist=tr.select_one('.albumtitle.ellipsis').text.strip()
artist

'쇼미더머니 10 Episode 2'

In [145]:
soup.find('albumtitle ellipsis')  # 앨범명

In [146]:
album=tr.select_one('.albumtitle.ellipsis').text.strip()
album

'쇼미더머니 10 Episode 2'

### 4. 한 페이지 내에 있는 정보 

1. 개별 리스트로 가져오기

In [147]:
# 어디에서 오류가 발생하는지 확인
rank_list, title_list, artist_list, album_list = [],[],[],[]
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:
        title = tr.select_one('.title.ellipsis').string.strip()
        artist = tr.select_one('.artist.ellipsis').text.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
    except:
        print(i)
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

16


In [148]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:            # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:         # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [149]:
lines = []
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:            # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:         # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line = {'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
    lines.append(line)

In [150]:
df = pd.DataFrame({'순위':rank_list, '제목':title_list,
                '아티스트':artist_list, '앨범':album_list})
df.head(10).tail(10)

,순위,제목,아티스트,앨범
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)
5,6,GANADARA (Feat. 아이유),박재범,GANADARA
6,7,취중고백,김민석 (멜로망스),취중고백
7,8,듣고 싶을까,MSG워너비 (M.O.M),듣고 싶을까
8,9,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
9,10,INVU,태연 (TAEYEON),INVU - The 3rd Album


In [151]:
df=pd.DataFrame(lines)
df.head(15).tail(5)

,순위,제목,아티스트,앨범
10,11,다정히 내 이름을 부르면,경서예지 & 전건호,다정히 내 이름을 부르면 (경서예지 x 전건호)
11,12,신호등,이무진,신호등
12,13,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
13,14,ELEVEN,IVE (아이브),ELEVEN
14,15,LOVE me,BE'O (비오),LOVE me


In [ ]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220415&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')

    for tr in trs:
        try:
            rank = int(tr.select_one('.number').get_text().split('\n')[0])
        except:
            rank = 0
        try:            # 일반적인 노래
            title = tr.select_one('.title.ellipsis').string.strip()
        except:         # 19금 노래
            title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').text.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        line = {'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
        lines.append(line)

In [153]:
df = pd.DataFrame(lines)
df.tail()

,순위,제목,아티스트,앨범
195,196,하루하루,BIGBANG (빅뱅),Stand Up (2008 빅뱅 3rd Mini Album)
196,197,우리의 이야기,멜로망스 (MeloMance),유미의 세포들 OST Part 8
197,198,운전만해 (We Ride),브레이브걸스 (Brave girls),We Ride
198,199,Don't Start Now,Dua Lipa,Future Nostalgia
199,200,Saturday Drip,NCT DREAM,Glitch Mode - The 2nd Album


In [154]:
df.to_csv('지니차트Top200.csv', index=False)